# AntiVirus
- 反病毒-AAA防护系统的第二个部分
- 可以识别出记录在案的病毒并自动处理
- 监控资源情况，判断是否被入侵，以发出警告

In [ ]:
import os
import json

# ---------------check ports-----------------
terminal = os.popen("/bin/busybox netstat -ltnp | awk '{ print $4; }'")
nr_ip = terminal.read()
nr_ip = nr_ip.split("\n")[2:-1]
terminal.close()

terminal = os.popen("/bin/busybox netstat -ltnp | awk '{ print $7; }'")
nr_pid = terminal.read()
nr_pid = nr_pid.split("\n")[2:-1]
terminal.close()

nr_result = {}
for i in range(0, len(nr_ip)):
    port = nr_ip[i].split(":")[-1]
    pid = nr_pid[i].split("/")[0]
    name = nr_pid[i].split("/")[1]
    nr_result[port] = {
        "name": name,
        "port": port,
        "pid": pid
    }
    
nr_ports = list(nr_result.keys())
standard_nr_ports = json.load(open("./standard_nr_ports.json", "r", encoding="utf-8"))

print(nr_result, "\n", nr_ports)

if nr_ports != standard_nr_ports:
    # kill process!
    print("EVENT-EMERGENCY: One/some disallowed ports are being listened! searching for the processes...")
    for port in nr_ports:
        if port not in standard_nr_ports:
            name = nr_result[port]["name"]
            pid = nr_result[port]["pid"]
            print("    Process-%s is listening the port-%s illegally, kill its pid-%s" % (name, port, pid))
            os.system("kill -9 " + pid)
else:
    print("EVENT-NORMAL: didn't find any abnormality with netstat")



In [48]:
import os
import time

# ---------------check %cpu-----------------
def get_cpu_usage_percent():
    def readCpuInfo():
        f = open('/proc/stat')
        lines = f.readlines();
        f.close()

        for line in lines:
            line = line.lstrip()
            counters = line.split()
            if len(counters) < 5:
                continue
            if counters[0].startswith('cpu'):
                break
        total = 0
        for i in range(1, len(counters)):
            total = total + float(counters[i])
        idle = float(counters[4])
        return {'total':total, 'idle':idle}

    def calcCpuUsage(counters1, counters2):

        idle = counters2['idle'] - counters1['idle']
        total = counters2['total'] - counters1['total']
        return 100 - (idle*100/total)

    counters1 = readCpuInfo()
    time.sleep(0.1)
    counters2 = readCpuInfo()
    result = calcCpuUsage(counters1, counters2)
    print(result)
    return result

cpu_usage = get_cpu_usage_percent()
if cpu_usage > 60:
    print("EVENT-EMERGENCY: high cpu used! check the program")
    
    tr = os.popen("/bin/busybox top -b -n 1").read()
    
    
    

7.4534161490683175


In [5]:
!/bin/busybox top -b -n 1 
CPU: 2  pid: 1 command: 9 cpu_every: 7

Mem: 62166672K used, 3466804K free, 566960K shrd, 381388K buff, 51449148K cached
CPU:   0% usr   0% sys   0% nic  98% idle   0% io   0% irq   0% sirq
Load average: 0.02 0.04 0.00 2/1893 1035784
  PID  PPID USER     STAT   VSZ %VSZ CPU %CPU COMMAND
284507284409 hadream  S    3704m   5%   3   0% /usr/bin/gnome-shell
724966 71237 www      S    3668m   5%   0   0% /www/server/apache/bin/httpd -k restart
725197 71237 www      S    2970m   4%   9   0% /www/server/apache/bin/httpd -k restart
724968 71237 www      S    2964m   4%   9   0% /www/server/apache/bin/httpd -k restart
724967 71237 www      S    2903m   4%   2   0% /www/server/apache/bin/httpd -k restart
 3973     1 root     S    2833m   4%   0   0% /usr/bin/Xvnc :1 -auth /root/.Xauthority -desktop hadream-server:1 (root) -fp catalogue:/etc/X11/fontpath.d -geometry 1024x768 -pn -rfbauth /root/.vnc/passwd -rfbport 5901 -rfbwait 30000
 1287     1 root     S    2647m   4%  12   0% /home/EasyDarwin-linux-8.1.0-1901141151/easydarwin
284723